Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [3]:
import pandas as pd
df_raw = pd.read_csv(DATA_PATH+'msrp.csv')

# Wrangle

In [4]:
from sklearn.impute import SimpleImputer

In [5]:
df_raw.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [6]:
## Wrangle function from Assign21
def wrangle(df):
    df = df.copy()
    
    ## Fix column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    ## Remove Outliers
    df = df[df['msrp'] <= 75000]
    
    ## Reduce high cardinality in 'make'
    df['make'] = [x if x in list(df['make'].value_counts().sort_values().tail(5).index) else 'Other' for x in df['make']]
    
    ## Reduce high cardinality in 
    
    ## Reduce high cardinality in 'market_category' (maybe remove NaN's)
    df['luxury'] = [x if type(x)==float else 1 if 'Luxury' in x else 0 for x in df['market_category']]
    
    ## 'Model' is going to allow for leakage in a way
    df.drop(columns=['model', 'market_category'], inplace=True)
    
    return df

In [7]:
df = wrangle(df_raw)

In [8]:
df.head()

,make,year,engine_fuel_type,engine_hp,engine_cylinders,transmission_type,driven_wheels,number_of_doors,vehicle_size,vehicle_style,highway_mpg,city_mpg,popularity,msrp,luxury
0,Other,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,26,19,3916,46135,1.0
1,Other,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28,19,3916,40650,1.0
2,Other,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,28,20,3916,36350,1.0
3,Other,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,28,18,3916,29450,1.0
4,Other,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28,18,3916,34500,1.0


In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [10]:
target = 'msrp'
y = df[target]
X = df.drop(columns=target)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
linreg = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

linreg.fit(X_train, y_train);

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['make', 'engine_fuel_type',
                                      'transmission_type', 'driven_wheels',
                                      'vehicle_size', 'vehicle_style'],
                                mapping=[{'col': 'make',
                                          'data_type': dtype('O'),
                                          'mapping': Other         1
Volkswagen    2
Ford          3
Dodge         4
Chevrolet     5
Toyota        6
NaN          -2
dtype: int64},
                                         {'col': 'engine_fuel_type',
                                          'data_type': dtype('O'),
                                          'mapping': regular unleaded                                 1
premium...
                                          'data_type': dtype('O'),
                                          'mapping': Wagon                   1
4dr SUV                 2
Convertible             

In [28]:
y_pred = [y_train.mean()]*len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))
print('Training MAE:', mean_absolute_error(y_train, linreg.predict(X_train)))
print('Test MAE:', mean_absolute_error(y_test, linreg.predict(X_test)))

Baseline MAE: 12429.751641196048
Training MAE: 5541.068739006571
Test MAE: 5529.234746267844
